# Preprocessing All Known Datasets

We explore how to preprocess our datasets for pretraining `roberthai`. 
* Thai text cleaning rules
* [Dynamic padding and co. to save training time](https://towardsdatascience.com/divide-hugging-face-transformers-training-time-by-2-or-more-21bf7129db9q-21bf7129db9e)
* [C4](https://arxiv.org/pdf/1910.10683.pdf)

In [1]:
%matplotlib inline
import json
import re
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
import dask.dataframe as dd
import dask
import multiprocessing
from thai2transformers.preprocess import process_transformers
from pythainlp.tokenize import word_tokenize, sent_tokenize
import matplotlib.pyplot as plt
nb_cores = multiprocessing.cpu_count()
nb_cores

8

## Source Files

Processed in this notebook and handed over in zip:

[classification_benchmarks](https://github.com/PyThaiNLP/classification-benchmarks)

* [prachathai-67k](https://github.com/PyThaiNLP/prachathai-67k)
* [wongnai-corpus](https://github.com/wongnai/wongnai-corpus/)
* [truevoice-intent](https://github.com/PyThaiNLP/truevoice-intent)
* [wisesight-sentimeit](https://github.com/PyThaiNLP/wisesight-sentiment)

[OPUS](http://opus.nlpl.eu/)


* [JW300](https://object.pouta.csc.fi/OPUS-JW300/v1/mono/th.txt.gz)
* [OpenSubtitles](https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/th.txt.gz)
* [QED](https://object.pouta.csc.fi/OPUS-QED/v2.0a/mono/th.txt.gz)
* [Tatoeba](https://object.pouta.csc.fi/OPUS-Tatoeba/v2020-05-31/mono/th.txt.gz)
* [bible-uedin](https://object.pouta.csc.fi/OPUS-bible-uedin/v1/mono/th.txt.gz)
* [tanzil](https://object.pouta.csc.fi/OPUS-Tanzil/v1/mono/th.txt.gz)

others

* [thwiki](https://dumps.wikimedia.org/thwiki/latest/)
* [thaipbs-111k](https://drive.google.com/file/d/1movJ5kjzhehD3OMtM9fPTS0VLaqd7oxK/view?usp=sharing)
* [thairath-222k](https://drive.google.com/file/d/1L5XatRRS1aekfuhMVu24SQtOCCUrhj_W/view?usp=sharing)
* [scb_mt_en_th_2020](https://github.com/vistec-AI/dataset-releases/releases/tag/scb-mt-en-th-2020_v1.0)
* [TNC]()


To be process on VISTEC server:
* [wisesight-large](gs://aires_thai2transformers/raw_data/wisesight.drop_dup.csv)
* [pantip-large](gs://aires_thai2transformers/data/raw_data/pantip_2019_2012.zip)

## Cleaning Rules

Pre rules
* lowercase
* fix_html
* replace_url
* rm_brackets
* replace_newlines
* rm_useless_spaces
* replace_rep_after

Tokenizer
* newmm

Post rules
* ungroup_emoji
* replace_wrep_post

Filter rules
* word count [5,300]

In [2]:
def process_one(text_list, min_seq_length=5, max_seq_length=300, sep_func=lambda x: x.split('\n')):
    word_counts = []
    texts = []
    for text in text_list:
        text = text.strip()
        word_count = len(word_tokenize(text))
        if word_count > max_seq_length:
            sub_text = [process_transformers(i) for i in sep_func(text)]
            sub_word_count = [len(word_tokenize(i)) for i in sub_text]
            texts+=sub_text
            word_counts+=sub_word_count
        else:
            texts.append(process_transformers(text))
            word_counts.append(word_count)
    df = pd.DataFrame({"text": texts, "wc": word_counts})
    print(np.percentile(df.wc, 5), np.percentile(df.wc, 95))
    df = df[(df.wc >= min_seq_length) & (df.wc <= max_seq_length)]
    return df.drop_duplicates().reset_index(drop=True)

## `pantip-large`

In [ ]:
# split -l 1000000 pantip_bk.json

In [2]:
%%time
def process_one_pantip(text_list, min_seq_length=5, max_seq_length=300, sep_func=sent_tokenize):
    word_counts = []
    texts = []
    for text in text_list:
        text = text.strip()
        word_count = len(word_tokenize(text))
        if word_count > max_seq_length:
            sub_text = [process_transformers(i) for i in sep_func(text)]
            sub_word_count = [len(word_tokenize(i)) for i in sub_text]
            texts+=sub_text
            word_counts+=sub_word_count
        else:
            texts.append(process_transformers(text))
            word_counts.append(word_count)
    return pd.DataFrame({"text": texts, "wc": word_counts})

def process_fname_pantip(fname, min_seq_length=5, max_seq_length=300, max_lines=1001):
    line_count=0
    reses=[]
    with open(fname,'r') as f:
        for line in tqdm(f):
            if line_count > max_lines: break  
            try:
                res = process_one_pantip([json.loads(line)['comment']])
                reses.append(res)
                line_count+=1
            except:
#                 print('Not comment')
                try:
                    res = process_one_pantip([json.loads(line)['content']])
                    reses.append(res)
                    line_count+=1
                except:
#                     print('Neither comment nor content')
                    print(json.loads(line))
    df = pd.concat(reses)
    df['text'] = df.text.map(lambda x: x.split('แก้ไขข้อความเมื่อ')[0])
    df = df[(df.wc >= min_seq_length) & (df.wc <= max_seq_length)]
    return df

fnames = glob.glob('raw_data/pantip-large/*')
df = process_fname_pantip(fnames[20],max_lines=1000)
x = list(df.sort_values('wc',ascending=False).text)


CPU times: user 2.96 s, sys: 52.7 ms, total: 3.02 s
Wall time: 2.96 s


In [ ]:
with multiprocessing.Pool(nb_cores) as pool:
    results = pool.map(process_fname_pantip, fnames[:3])

In [ ]:
result_df = pd.concat(results).drop_duplicates().reset_index(drop=True)
pd.Series(result_df.text.unique()).to_csv('cleaned_data/pantip-large/pantip-large.txt',index=False,header=None)

In [ ]:
with open('cleaned_data/pantip-large/pantip-large.txt','r')

## `prachathai-67k`

Processed in this notebook and saved to `cleaned_data/prachathai-67k-cleaned`

In [33]:
# #download prachathai-67k.zip and extract .csv to raw_data; also created cleaned_data/prachathai-67k-cleaned
# !wget https://www.dropbox.com/s/fsxepdka4l2pr45/prachathai-67k.zip; unzip prachathai-67k.zip; mv prachathai-67k.csv raw_data; rm prachathai-67k.zip
# !mkdir cleaned_data/prachathai-67k-cleaned

In [2]:
df = pd.read_csv('raw_data/prachathai-67k.csv')
df_d = dd.from_pandas(df,npartitions=nb_cores*10)
df_d.shape

(Delayed('int-bebd4fa7-043b-4354-ae3a-a68b9b1c9c73'), 5)

In [3]:
%%time
#count tokens to see if we can just save titles
df_d['title_tokens'] = df_d.title.map(lambda x: len(word_tokenize(x)))
df = df_d.compute()

CPU times: user 12.8 s, sys: 283 ms, total: 13.1 s
Wall time: 12.6 s


In [4]:
#yes we can!
df.title_tokens.describe()

count    67889.000000
mean        15.876755
std          5.752439
min          1.000000
25%         12.000000
50%         15.000000
75%         20.000000
max         76.000000
Name: title_tokens, dtype: float64

In [63]:
save_df = process_one(df.title)


7.0 26.0


In [65]:
save_df.wc.describe()

count    67339.000000
mean        15.976581
std          5.662981
min          5.000000
25%         12.000000
50%         16.000000
75%         20.000000
max         76.000000
Name: wc, dtype: float64

In [66]:
save_df.text.to_csv('cleaned_data/prachathai-67k-cleaned/prachathai-67k_title.txt',header=None,index=False)

In [5]:
#see number of words once body_text is split by \n
[len(word_tokenize(i)) for i in df.body_text[1].split('\n')]

[37, 43, 33, 35, 59, 46, 91, 52, 73, 62, 61, 12, 4, 0]

In [6]:
#save the body_text to raw_data/prachathai-67k
save_df = process_one(df.body_text)
save_df.text.to_csv('cleaned_data/prachathai-67k-cleaned/prachathai-67k_body.txt',header=None,index=False)

NameError: name 'process_one' is not defined

In [ ]:
save_df.wc.describe()

In [8]:
with open('cleaned_data/prachathai-67k-cleaned/prachathai-67k_body.txt','r') as f:
    x = f.readlines()

## `wongnai`

In [7]:
#get data
# !wget https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip
# !mkdir raw_data/wongnai-corpus
# !unzip review_dataset.zip -d raw_data/wongnai-corpus

In [11]:
train = pd.read_csv('raw_data/wongnai-corpus/w_review_train.csv',header=None,sep=';')
test = pd.read_csv('raw_data/wongnai-corpus/test_file.csv',sep=';')
train.shape, test.shape

((40000, 2), (6203, 2))

In [18]:
df = pd.concat([train.iloc[:,0],test.iloc[:,1]]).reset_index(drop=True)
df

0        ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1        อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...
2        ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...
3        รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...
4        อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...
                               ...                        
46198    ร้านโรตีสายไหมพญาวัง อยู่ถนนรัตนโกสินทร์ หลังโ...
46199    ข้าวผัดคอหมูย่าง ข้าวผัดมากลิ่นหอม คอหมูย่างอร...
46200    ร้านอาหารมีหลายโซน แนะนำให้เดินดูก่อนค่อยเลือก...
46201         ขนมปังร้านในเครือยามาซากิ จริงๆร้านนี้เปิ...
46202    ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
Length: 46203, dtype: object

In [51]:
save_df = process_one(list(df))


0.0 189.0


In [53]:
save_df.wc.describe()

count    101451.000000
mean         72.514919
std          61.851462
min           5.000000
25%          23.000000
50%          60.000000
75%          99.000000
max         300.000000
Name: wc, dtype: float64

In [57]:
save_df.text.to_csv('cleaned_data/wongnai-corpus-cleaned/wongnai-corpus-cleaned.txt',header=None,index=False)

## `truevoice-intent`

In [14]:
# !mkdir cleaned_data/truevoice-intent-cleaned
# !mkdir raw_data/truevoice-intent
# !wget https://github.com/PyThaiNLP/truevoice-intent/raw/master/mari-intent.zip; unzip mari-intent -d raw_data/truevoice-intent

In [15]:
train = pd.read_csv('raw_data/truevoice-intent/mari-train-balanced.csv')
test = pd.read_csv('raw_data/truevoice-intent/mari-test-balanced.csv')
df = pd.concat([train,test]).reset_index(drop=True)
df

,texts,texts_deepcut,action,object,destination
0,สอบถาม package internet ว่ามีแบบไหนบ้างครับ,สอบถาม package internet ว่า มี แบบ ไหน บ้าง ครับ,enquire,promotion,promotions
1,พอดีพี่ซื้อเบอร์ซิม Inter ไปแล้ว แต่ยังไม่ได้ล...,พอดี พี่ ซื้อ เบอร์ซิม Inter ไป แล้ว แต่ ยัง ไ...,enquire,detail,billing and payment
2,ค่ะ พี่ค่ะ คืออยู่ๆ เน็ตเล่นไม่ได้อ่ะค่ะ,ค่ะ พี่ ค่ะ คือ อยู่ ๆ เน็ต เล่น ไม่ ได้ อ่ะ ค่ะ,enquire,internet,internet
3,การเปลี่ยน ระบบจาก ทรู เป็น ทรูเอช มีค่าบริการ...,การ เปลี่ยน ระบบ จาก ทรู เป็น ทรูเอช มี ค่า บร...,enquire,rate,promotions
4,เช็คให้หน่อยได้ไหมครับ ว่าเบอร์ของผม เปลี่ยนเป...,เช็ค ให้ หน่อย ได้ ไหม ครับ ว่า เบอร์ ของ ผม เ...,enquire,package,promotions
...,...,...,...,...,...
12905,จะสอบถามเรื่องค่าโทรศพท์ของเดือน มกราคม,จะ สอบถาม เรื่อง ค่า โทรศพท์ ของ เดือน มกราคม,enquire,balance,billing and payment
12906,ครับ ต้องการเปลี่ยนโปรโมชั่นแพ็คเกจของทรูมูฟ เ...,ครับ ต้องการ เปลี่ยน โปรโมชั่น แพ็คเกจ ของ ทรู...,change,package,promotions
12907,ผมต้องการดูฟุตบอลในมือถือ ผมต้องทำยังไงคครับ,ผม ต้องการ ดู ฟุตบอล ใน มือ ถือ ผม ต้อง ทำ ยัง...,enquire,service,other queries
12908,สอบถามโปรที่ใช้อยู่มีบริการฟรีอะไรบ้าง,สอบถาม โปร ที่ ใช้ อยู่ มี บริการ ฟรี อะไร บ้าง,enquire,detail,billing and payment


In [18]:
save_df = process_one(list(df.texts))


4.0 31.0


In [19]:
save_df.wc.describe()

count    12006.000000
mean        14.448609
std          9.135838
min          5.000000
25%          8.000000
50%         12.000000
75%         18.000000
max        118.000000
Name: wc, dtype: float64

In [20]:
save_df.text.to_csv('cleaned_data/truevoice-intent-cleaned/truevoice-intent-cleaned.txt',header=None,index=False)

## `wisesight-sentiment`

In [22]:
# !mkdir raw_data/wisesight-sentiment
# !mkdir cleaned_data/wisesight-sentiment-cleaned
# !wget https://github.com/PyThaiNLP/wisesight-sentiment/archive/master.zip; unzip master.zip -d raw_data/wisesight-sentiment

In [23]:
with open('raw_data/wisesight-sentiment/wisesight-sentiment-master/neg.txt','r') as f: neg = f.readlines()
with open('raw_data/wisesight-sentiment/wisesight-sentiment-master/neu.txt','r') as f: neu = f.readlines()
with open('raw_data/wisesight-sentiment/wisesight-sentiment-master/pos.txt','r') as f: pos = f.readlines()
with open('raw_data/wisesight-sentiment/wisesight-sentiment-master/q.txt','r') as f: q = f.readlines()
all_msg = neg + neu + pos + q
len(all_msg)

26737

In [24]:
save_df = process_one(list(all_msg))


2.0 99.0


In [25]:
save_df.wc.describe()

count    21844.000000
mean        30.088354
std         37.568328
min          5.000000
25%          8.000000
50%         16.000000
75%         35.000000
max        300.000000
Name: wc, dtype: float64

In [27]:
save_df.text.to_csv('cleaned_data/wisesight-sentiment-cleaned/wisesight-sentiment.txt',header=None,index=False)

## `thaipbs-111k`

## `thairath-222k`

## `thwiki`

Run `prepare_wiki_th.sh`.

In [19]:
with open('raw_data/thwiki/wiki/th-all/th.wiki.test.tokens') as f:
    test_df = process_one(list(f.readlines()))
with open('raw_data/thwiki/wiki/th-all/th.wiki.valid.tokens') as f:
    valid_df = process_one(list(f.readlines()))
with open('raw_data/thwiki/wiki/th-all/th.wiki.train.tokens') as f:
    train_df = process_one(list(f.readlines()))
df = pd.concat([test_df,valid_df,train_df]).reset_index(drop=True)


0.0 149.0



0.0 157.0



0.0 142.0


In [20]:
df.text.to_csv('cleaned_data/thwiki-cleaned/thwiki.txt',header=None,index=False)

In [21]:
with open('cleaned_data/thwiki-cleaned/thwiki.txt','r') as f:
    x = f.readlines()
len(x),x[:10]

(842552,
 ['ปลาบู่รำไพ\n',
  '""\n',
  'ปลาบู่รำไพ ปลาน้ำจืดและน้ำกร่อยขนาดเล็กชนิดหนึ่ง ในวงศ์ปลาบู่ (gobiidae) มีลำตัวยาวทรงกระบอก หัวเล็ก ปากกว้างในตัวผู้ เกล็ดค่อนข้างใหญ่ ลำตัวมีสีคล้ำหรือสีน้ำตาล ขอบเกล็ดมีแต้มสีคล้ำหลังช่องเหงือกด้านบนมีดวงรีสีดำและขอบสีจาง ครีบหลังมีลายสีดำและขอบสีจาง ครีบหางมีลายเส้นสีคล้ำ\n',
  '"มีขนาดโตเต็มที่ประมาณ 3-6 เซนติเมตร มีพฤติกรรมชอบอยู่รวมกลุ่มกันเป็นฝูงใหญ่ใต้ท้องน้ำ พบกระจายพันธุ์อยู่ในอินเดีย, พม่า, ไทย, มาเลเซียจนถึงอินโดนีเซีย ในบริเวณปากแม่น้ำที่เป็นป่าจากหรือเป็นแหล่งน้ำกร่อย ในประเทศไทยพบได้ในพื้นที่ภาคกลางและภาคใต้ทั้งฝั่งตะวันตก ส่วนฝั่งตะวันออกพบเฉพาะจังหวัดสุราษฎร์ธานีและนครศรีธรรมราช"\n',
  'ปลาบู่รำไพ ถูกค้นพบครั้งแรกในคลองสายหนึ่งในพื้นที่กรุงเทพมหานคร และถูกอนุกรมวิธานโดย ดร.ฮิว แมคคอร์มิค สมิธ อธิบดีกรมประมงคนแรก และได้มีการตั้งชื่อเพื่อเป็นการถวายพระเกียรติแด่ สมเด็จพระนางเจ้ารำไพพรรณี พระอัครมเหสีในพระบาทสมเด็จพระปกเกล้าเจ้าอยู่หัว รัชกาลที่ 7 ณ สยามสมาคม ปัจจุบันเป็นปลาที่แนวโน้มใกล้สูญพันธุ์ และไม่ทราบสถานะที่แน่ชัดว่าจะยังมี

## `JW300`

In [ ]:
# # !mkdir raw_data/jw300; mkdir cleaned_data/jw300-cleaned
# !wget https://object.pouta.csc.fi/OPUS-JW300/v1/mono/th.txt.gz; gunzip th.txt.gz -d raw_data/opensub
# !mv th.txt raw_data/jw300

In [30]:
with open('raw_data/jw300/th.txt') as f:
    x = [i.replace(' \u200b ','') for i in  f.readlines()]
    df = process_one(x)
df


4.0 54.0


,text,wc
0,คนอื่นคิดอย่างไรกับเราเป็นเรื่องสำคัญไหม?,9
1,แทบทุกคนชอบที่ได้รับการยกย่อง.,6
2,คำชมเชยทำให้เรามีความสุขและรู้สึกว่าประสบความส...,9
3,ความต้องการที่จะได้รับการยอมรับจากผู้อื่นทำให้...,15
4,แต่เราจะรู้สึกต่างออกไปอย่างสิ้น เชิง เมื่อรู้...,20
...,...,...
726735,เขาตอบด้วยภาษาอาหรับที่ชัดถ้อยชัดคำว่า “เปล่าค...,29
726736,ผมจะเป็นพยานพระยะ โฮ วาที่บ้าน และถืออีกศาสนาห...,27
726737,ผลก็คือ เขาได้รับการยกเว้น.,10
726738,หลังจากที่ได้รับบัพ ติ สมาแล้ว ชายชาวเอธิโอเปี...,28


In [33]:
df.wc.describe()

count    726740.000000
mean         25.600751
std          18.172516
min           5.000000
25%          14.000000
50%          22.000000
75%          32.000000
max         298.000000
Name: wc, dtype: float64

In [34]:
df.text.to_csv('cleaned_data/jw300-cleaned/jw300.txt',header=None,index=False)

## `OpenSubitles`

In [39]:
!mkdir raw_data/opensubtitles; mkdir cleaned_data/opensubtitles-cleaned
!wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/th.txt.gz; gunzip th.txt.gz
!mv th.txt raw_data/opensubtitles

mkdir: cannot create directory ‘raw_data/opensubtitles’: File exists
mkdir: cannot create directory ‘cleaned_data/opensubtitles-cleaned’: File exists
--2020-08-26 07:51:57--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/th.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140139092 (134M) [application/gzip]
Saving to: ‘th.txt.gz’

th.txt.gz           100%[===================>] 133.65M  10.5MB/s    in 14s     

2020-08-26 07:52:13 (9.26 MB/s) - ‘th.txt.gz’ saved [140139092/140139092]



In [44]:
with open('raw_data/opensubtitles/th.txt') as f:
    x = f.readlines()
df = process_one(x)
df


1.0 15.0


,text,wc
0,นี่ ซุนกึมชิอา...,7
1,อาจุมม่า (อาจุมม่า,8
2,- เรียกหญิงที่อายุเยอะกว่าหรือป้าก็ได้),12
3,แกมีเงินเท่าไหร่ล่ะ?,5
4,พ่อบอกว่าเขารู้จุดอ่อนของหนู พ่อคิดว่าเขาจะมาห...,20
...,...,...
5355810,และหนูพูดถึงสถานที่ที่หนูไปมา แต่ไม่มีใครเชื่อ...,16
5355811,-เร็วเข้า เราต้องเอาหนูออกไปจากที่นี่,12
5355812,เสียงกรีดร้องนั่นของใคร,5
5355813,แล้วน้ำที่เหลือหายไปไหนหมดล่ะ?,10


In [45]:
df.wc.describe()

count    5.355815e+06
mean     9.301821e+00
std      3.894055e+00
min      5.000000e+00
25%      6.000000e+00
50%      8.000000e+00
75%      1.100000e+01
max      2.450000e+02
Name: wc, dtype: float64

In [46]:
df.text.to_csv('cleaned_data/opensubtitles-cleaned/opensubtitles.txt',header=None,index=False)

In [3]:
with open('cleaned_data/opensubtitles-cleaned/opensubtitles.txt','r') as f:
    x = f.readlines()
x[:10], len(x)

(['นี่ ซุนกึมชิอา...\n',
  'อาจุมม่า (อาจุมม่า\n',
  '- เรียกหญิงที่อายุเยอะกว่าหรือป้าก็ได้)\n',
  'แกมีเงินเท่าไหร่ล่ะ?\n',
  'พ่อบอกว่าเขารู้จุดอ่อนของหนู พ่อคิดว่าเขาจะมาหาประโยชน์จากมันใช่ไหม?\n',
  'แม้กระทั่งหน้าร้อนหนูก็ไม่เคยได้กินไอติมเลย เพราะว่าอยากจะเก็บเงินเอาไว้\n',
  'และก็เหมือนที่พ่อทำ หนูเอาชนะไพ่โกสต็อบได้บ่อย\n',
  'ไม่ว่ายังไงหนูก็เป็นลูกพ่อนะ\n',
  'ล็อตเตอร์รี่ที่หนูซื้อนั้นก็เหมือนกัน...\n',
  'ไม่ คุณพ่อครับ!\n'],
 5355815)

## `QED`

In [4]:
!mkdir raw_data/qed; mkdir cleaned_data/qed-cleaned
!wget https://object.pouta.csc.fi/OPUS-QED/v2.0a/mono/th.txt.gz; gunzip th.txt.gz
!mv th.txt raw_data/qed

--2020-08-26 12:07:52--  https://object.pouta.csc.fi/OPUS-QED/v2.0a/mono/th.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9704104 (9.3M) [application/gzip]
Saving to: ‘th.txt.gz’

th.txt.gz           100%[===================>]   9.25M  3.66MB/s    in 2.5s    

2020-08-26 12:07:57 (3.66 MB/s) - ‘th.txt.gz’ saved [9704104/9704104]



In [5]:
with open('raw_data/qed/th.txt') as f:
    x = f.readlines()
df = process_one(x)
df.wc.describe()


3.0 20.0


,text,wc
0,เริ่มด้วยการเขียน 0.8 ให้อยู่ในรูปเศษส่วน,11
1,ซึ่ง 0.8 เลข 8 อยู่ตรงนี้,10
2,อยู่ที่ตำแหน่งหนึ่งในสิบ มันคือหนึ่งในสิบ ซึ่ง...,19
3,8 ในสิบและเราสามารถเขียนให้ตรงตามตัวอักษรว่า,13
4,มันเท่ากีบ 8 ในสิบ,8
...,...,...
406552,ว่าพวกเรากำลังออกแบบเพื่อโลกที่กำลังจะมาถึง,9
406553,ผมเข้าสู่ธุรกิจการออกแบบผลิตภัณฑ์นี้,5
406554,และผมได้เรียนรู้ว่า ถ้าคุณต้องการที่จะสร้างโลก...,16
406555,คุณต้องออกแบบเพื่อผลลัพธ์,5


In [6]:
df.text.to_csv('cleaned_data/qed-cleaned/qed.txt',header=None,index=False)

In [7]:
with open('cleaned_data/qed-cleaned/qed.txt','r') as f:
    x = f.readlines()
x[:10], len(x)

(['เริ่มด้วยการเขียน 0.8 ให้อยู่ในรูปเศษส่วน\n',
  'ซึ่ง 0.8 เลข 8 อยู่ตรงนี้\n',
  'อยู่ที่ตำแหน่งหนึ่งในสิบ มันคือหนึ่งในสิบ ซึ่งคุณสามารถอ่านได้ว่า\n',
  '8 ในสิบและเราสามารถเขียนให้ตรงตามตัวอักษรว่า\n',
  'มันเท่ากีบ 8 ในสิบ\n',
  'หรือ 8 ส่วน 10 และในตอนนี้ พวกเราได้เขียนมันออกมา\n',
  'ในรูปของเศษส่วน และถ้าเราต้องการลดทอนเศษส่วนลง\n',
  'ทั้ง 8 และ 10 มีตัวประกอบร่วมกัน ซึ่งต่างก็ถูกหารด้วย 2 ได้\n',
  'ดังนั้นเริ่มหารทั้งตัวเศษและตัวส่วนด้วย 2 พวกเราไม่สามารถเปลี่ยนค่าของเศษส่วนได้ เพราะว่าเราหารตัวเศษและตัวส่วนด้วยสิ่งเดียวกัน\n',
  '8 หารด้วย 2 ได้ 4\n'],
 406557)

## `tatoeba`

In [8]:
!mkdir raw_data/tatoeba; mkdir cleaned_data/tatoeba-cleaned
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v2020-05-31/mono/th.txt.gz; gunzip th.txt.gz
!mv th.txt raw_data/tatoeba

--2020-08-26 12:09:47--  https://object.pouta.csc.fi/OPUS-Tatoeba/v2020-05-31/mono/th.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44530 (43K) [application/gzip]
Saving to: ‘th.txt.gz’

th.txt.gz           100%[===================>]  43.49K   153KB/s    in 0.3s    

2020-08-26 12:09:48 (153 KB/s) - ‘th.txt.gz’ saved [44530/44530]



In [9]:
with open('raw_data/tatoeba/th.txt') as f:
    x = f.readlines()
df = process_one(x)
df.text.to_csv('cleaned_data/tatoeba-cleaned/tatoeba.txt',header=None,index=False)
df.wc.describe()


2.0 10.0


count    1791.000000
mean        7.028476
std         2.828580
min         5.000000
25%         5.000000
50%         6.000000
75%         8.000000
max        47.000000
Name: wc, dtype: float64

In [11]:
with open('cleaned_data/tatoeba-cleaned/tatoeba.txt','r') as f:
    x = f.readlines()
x[:10], len(x)

(['ภาษาไทยเป็นภาษาราชการในประเทศไทย\n',
  'ภาษาเดียวไม่เคยพอ\n',
  'เมื่อไหร่จะมีการนำเที่ยวอีก\n',
  'ผมรักคุณมากกว่าที่คุณรักผม\n',
  'เธอดูตื่นเต้นสุดๆเลย\n',
  'ฉันคิดว่ามันเป็นวิธีที่ดีที่สุด\n',
  'ถึงเขาจะขอโทษ ฉันก็ยังโกรธ\n',
  'ถึงเขาจะขอโทษแล้ว แต่ฉันก็ยังโกรธ\n',
  'เมื่อวานอากาศโคตรร้อนเลย\n',
  'อย่าถูกเนื้อต้องตัวผู้หญิงที่ท่านไม่รู้จักดี\n'],
 1791)

 ## `bible-uedin`

In [12]:
!mkdir raw_data/bibleuedin; mkdir cleaned_data/bibleuedin-cleaned
!wget https://object.pouta.csc.fi/OPUS-bible-uedin/v1/mono/th.txt.gz; gunzip th.txt.gz
!mv th.txt raw_data/bibleuedin

--2020-08-26 12:12:31--  https://object.pouta.csc.fi/OPUS-bible-uedin/v1/mono/th.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3801461 (3.6M) [application/gzip]
Saving to: ‘th.txt.gz’

th.txt.gz           100%[===================>]   3.62M  1.82MB/s    in 2.0s    

2020-08-26 12:12:35 (1.82 MB/s) - ‘th.txt.gz’ saved [3801461/3801461]



In [13]:
with open('raw_data/bibleuedin/th.txt') as f:
    x = f.readlines()
df = process_one(x)
df.text.to_csv('cleaned_data/bibleuedin-cleaned/tatoeba.txt',header=None,index=False)
df.wc.describe()


16.0 103.0


count    61708.000000
mean        47.874781
std         27.374443
min          5.000000
25%         28.000000
50%         41.000000
75%         62.000000
max        209.000000
Name: wc, dtype: float64

In [14]:
with open('cleaned_data/bibleuedin-cleaned/tatoeba.txt','r') as f:
    x = f.readlines()
x[:10], len(x)

(['ในเริ่มแรกนั้นพระเจ้าทรงเนรมิตสร้างฟ้าและแผ่นดินโลก\n',
  'แผ่นดินโลกนั้นก็ปราศจากรูปร่างและว่างเปล่าอยู่ ความมืดอยู่เหนือผิวน้ำ และพระวิญญาณของพระเจ้าปกอยู่เหนือผิวน้ำนั้น\n',
  '"วันที่หนึ่งปรากฏมีความสว่างเกิดขึ้นพระเจ้าตรัสว่า ""จงให้มีความสว่าง"" แล้วความสว่างก็เกิดขึ้น"\n',
  'พระเจ้าทรงเห็นว่าความสว่างนั้นดี และพระเจ้าทรงแยกความสว่างนั้นออกจากความมืด\n',
  'พระเจ้าทรงเรียกความสว่างนั้นว่าวัน และพระองค์ทรงเรียกความมืดนั้นว่าคืน มีเวลาเย็นและเวลาเช้าเป็นวันที่หนึ่ง\n',
  '"วันที่สองมีเมฆปรากฏอยู่เหนือผิวน้ำนั้นพระเจ้าตรัสว่า ""จงให้มีพื้นอากาศในระหว่างน้ำ และจงให้พื้นอากาศนั้นแยกน้ำออกจากน้ำ"""\n',
  'พระเจ้าทรงสร้างพื้นอากาศ และทรงแยกน้ำซึ่งอยู่ใต้พื้นอากาศจากน้ำซึ่งอยู่เหนือพื้นอากาศ ก็เป็นดังนั้น\n',
  'พระเจ้าทรงเรียกพื้นอากาศว่าฟ้า มีเวลาเย็นและเวลาเช้าเป็นวันที่สอง\n',
  '"วันที่สามปรากฏว่ามีทะเล แผ่นดินและพืชพันธุ์ต่างๆพระเจ้าตรัสว่า ""จงให้น้ำที่อยู่ใต้ฟ้ารวบรวมเข้าอยู่แห่งเดียวกัน และจงให้ที่แห้งปรากฏขึ้น"" ก็เป็นดังนั้น"\n',
  'พระเจ้าทรงเรียกที่แห้งว่าแผ่นดิน และที่น

## `tanzil`

In [15]:
!mkdir raw_data/tanzil; mkdir cleaned_data/tanzil-cleaned
!wget https://object.pouta.csc.fi/OPUS-Tanzil/v1/mono/th.txt.gz; gunzip th.txt.gz
!mv th.txt raw_data/tanzil

--2020-08-26 12:13:34--  https://object.pouta.csc.fi/OPUS-Tanzil/v1/mono/th.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423484 (414K) [application/gzip]
Saving to: ‘th.txt.gz’

th.txt.gz           100%[===================>] 413.56K   294KB/s    in 1.4s    

2020-08-26 12:13:36 (294 KB/s) - ‘th.txt.gz’ saved [423484/423484]



In [16]:
with open('raw_data/tanzil/th.txt') as f:
    x = f.readlines()
df = process_one(x)
df.text.to_csv('cleaned_data/tanzil-cleaned/tanzil.txt',header=None,index=False)
df.wc.describe()


8.0 89.25


count    6135.000000
mean       37.161206
std        27.886302
min         5.000000
25%        17.000000
50%        30.000000
75%        49.000000
max       217.000000
Name: wc, dtype: float64

In [17]:
with open('cleaned_data/tanzil-cleaned/tatoeba.txt','r') as f:
    x = f.readlines()
x[:10], len(x)

(['ด้วยพระนามของอัลลอฮฺ ผู้ทรงกรุณาปราณี ผู้ทรงเมตตาเสมอ\n',
  'การสรรเสริญทั้งหลายนั้น เป็นสิทธิของอัลลอฮฺผู้เป็นพระเจ้าแห่งสากลโลก\n',
  'ผู้ทรงกรุณาปราณี ผู้ทรงเมตตาเสมอ\n',
  'ผู้ทรงอภิสิทธิ์แห่งวันตอบแทน\n',
  'เฉพาะพระองค์เท่านั้นที่พวกข้าพระองค์เคารพอิบาดะฮฺ และเฉพาะพระองค์เท่านั้นที่พวกข้าพระองค์ขอความช่วยเหลือ\n',
  'ขอพระองค์ทรงแนะนำพวกข้าพระองค์ซึ่งทางอันเที่ยงตรง\n',
  '(คือ) ทางของบรรดาผู้ที่พระองค์ได้ทรงโปรดปราณแก่พวกเขา มิใช่ในทางของพวกที่ถูกกริ้ว และมิใช่ทางของพวกที่หลงผิด\n',
  'อะลิฟ ลาม มีม\n',
  'คัมภีร์นี้ ไม่มีความสงสัยใด ๆ ในนั้น เป็นคำแนะนำสำหรับบรรดาผู้ยำเกรงเท่านั้น\n',
  'คือบรรดาผู้ศรัทธาต่อสิ่งเร้นลับ และดำรงไว้ซึ่งการละหมาด และส่วนหนึ่งจากสิ่งที่เราได้ให้เป็นปัจจัยยังชีพแก่พวกเขานั้น พวกเขาก็บริจาค\n'],
 6135)

## `thairath-222k`

In [3]:
# !mkdir raw_data/thairath; mkdir cleaned_data/thairath-cleaned
# !wget https://www.dropbox.com/s/99bzumhy9ahjemn/thairath-222_final.csv; mv thairath-222_final.csv raw_data/thairath

In [5]:
thairath_df = pd.read_csv('raw_data/thairath/thairath-222_final.csv')
thairath_df.head()

,title,body,summary,labels,tags,url,date
0,"โป้งเดียวร่วง ""ฝนพันล้าน"" ต่อยแจ๊คแพ้น็อก 4 มว...",วันที่ 30 พ.ย. 62 ศึกมวยไทยเกียรติเพชร ซูเปอร...,ฝนพันล้าน พี.เค.แสนชัยมวยไทยยิม หมัดหนักเอาเรื...,"กีฬา,มวย/MMA","มวยลุมพินี,มวยไทย,ฝนพันล้าน พี.เค.แสนชัยมวยไทย...",https://www.thairath.co.th/sport/boxing-mma/17...,2019-12-01T00:30:00+07:00
1,"""สุรศักดิ์มนตรี"" ผงาดแชมป์โต๊ะเล็กเอสแบค-นอร์ท...","กรุงเทพ ฟุตซอลนักเรียน คัพ ครั้งที่ 14,วันที่ ...",ร.ร.สุรศักดิ์มนตรีชนะจุดโทษคว้าแชมป์รุ่นอายุไม...,"กีฬา,ฟุตซอล","ฟุตซอล,ฟุตซอลเยาวชน,เอสแบค-นอร์ทกรุงเทพ ฟุตซอล...",https://www.thairath.co.th/sport/futsal/1715406,2019-12-01T01:58:00+07:00
2,หนุ่มไทใหญ่ชะตาขาด จยย.เสียหลักล้มดับ คาดถูกรถ...,เมื่อคืนวันที่ 30 พ.ย. 2562 เกิดอุบัติเหตุรถจั...,เกิดเหตุหนุ่มชาวไทใหญ่รถจักรยานยนต์ล้มเสียชีวิ...,"ข่าว,ทั่วไทย","รถทับซ้ำ,จักรยานยนต์ล้ม,ไทใหญ่,เชียงใหม่,อุบัต...",https://www.thairath.co.th/news/local/north/17...,2019-12-01T02:01:00+07:00
3,"""ปาซาลิช"" กดสองเม็ด ""อตาลันตา"" บุกทุบ ""เบรสชา""...",การแข่งขันศึกฟุตบอลกัลโช เซเรีย อา อิตาลี ฤดูก...,มาริโอ ปาซาลิช เหมาคนเดียวสองประตู ช่วยให้ อตา...,"กีฬา,ฟุตบอลยุโรป","ผลบอล,กัลโช เซเรีย อา,เบรสชา,อตาลันตา,ข่าวบอล,...",https://www.thairath.co.th/sport/eurofootball/...,2019-12-01T01:35:00+07:00
4,"ไม่สนตารางคะแนน ""เป๊ป"" ยันแมนฯ ซิตี้โฟกัสผลงาน...",สำนักข่าวต่างประเทศรายงานวันที่ 1 ธ.ค.ว่า เป๊ป...,นายใหญ่ทีมเรือใบสีฟ้ายอมรับ ลูกทีมทำทุกอย่างแล...,"กีฬา,ฟุตบอลยุโรป","เป๊ป กวาร์ดิโอลา,แมนเชสเตอร์ ซิตี้,พรีเมียร์ลี...",https://www.thairath.co.th/sport/eurofootball/...,2019-12-01T03:50:00+07:00


In [19]:
df = process_one(list(thairath_df.title))
df.text.to_csv('cleaned_data/thairath-cleaned/thairath_title.txt',header=None,index=False)
df.wc.describe()


6.0 25.0


count    216421.000000
mean         17.680096
std           5.083441
min           5.000000
25%          15.000000
50%          18.000000
75%          21.000000
max          43.000000
Name: wc, dtype: float64

In [20]:
df = process_one(list(thairath_df.summary))
df.text.to_csv('cleaned_data/thairath-cleaned/thairath_summary.txt',header=None,index=False)
df.wc.describe()


27.0 63.0


count    222389.000000
mean         44.977211
std          11.096862
min           8.000000
25%          38.000000
50%          45.000000
75%          53.000000
max         174.000000
Name: wc, dtype: float64

In [6]:
df = process_one(list(thairath_df.body),sep_func=sent_tokenize)
df.text.to_csv('cleaned_data/thairath-cleaned/thairath_body.txt',header=None,index=False)
df.wc.describe()


3.0 68.0


count    4.941166e+06
mean     2.625106e+01
std      3.038816e+01
min      5.000000e+00
25%      1.000000e+01
50%      1.700000e+01
75%      3.100000e+01
max      3.000000e+02
Name: wc, dtype: float64

In [7]:
with open('cleaned_data/thairath-cleaned/thairath_body.txt','r') as f:
    x = f.readlines()

In [8]:
len(x),x[:10]

(4941186,
 ['วันที่ 30 พ.ย. 62\xa0 ศึกมวยไทยเกียรติเพชร ซูเปอร์ไฟต์ ที่สนามมวยเวทีลุมพินี รามอินทรา คู่เอก แจ๊ค ลูกมะขามหวาน\n',
  '"พบกับ ฝนพันล้าน พี.เค.แสนชัยมวยไทยยิม,แจ๊ค"\n',
  'ฝ่ายแดง ได้เปรียบรูปร่างก็จริง แต่เจอ ฝนพันล้าน พี.เค.แสนชัยมวยไทยยิม ฝ่ายน้ำเงิน ควงหมัดไล่ต่อยลำตัวสลับใบหน้า ตามด้วยเตะขา จนแจ๊คออกอาการไม่ดี\n',
  'ก่อนเกมจะจบลงแบบเด็ดขาดในยกที่ 4 เมื่อฝนพันล้านต่อยซ้ายหมัดเดียว\n',
  '"ส่งแจ๊คลงไปนอนแพ้น็อกไปในยกที่ 4 นี้เอง,ส่วนผลคู่อื่นมีดังนี้\xa0,\xa0\xa0\xa0\xa0\xa0\xa0 \xa0,ยอดพนมทวน ศิษย์นายกอนงค์ ชนะคะแนน"\n',
  '"ลูกตากสิน ส.ส.ศิริพงษ์ \xa0,วรพล เกียรติชัชนันท์ ชนะคะแนน ยอดมีชัย เกียรติจำรูญ,เหนือธรณี"\n',
  'สามชัยวิเศษสุก ชนะคะแนน ศักดิ์ชัยน้อย\n',
  '"ชนะน็อกยก 4 เอกทวีคูณ หมูปิ้งอร่อยจุงเบย,พระแสงเล็ก"\n',
  '"ส.โชคมีชัย ชนะคะแนน ซุปเปอร์เจ๋ง ศิษย์ครูเยี่ยม,หนึ่งปฐพี"\n',
  '"ร.ร.กีฬาจังหวัดลำปาง ชนะคะแนน เพชรเสกสรร ทุ่งปาหนันยางทอง,โรเบิร์ต ศ.ธนบวร ชนะคะแนน พลอยรุ่งเพชร ทีเด็ด99\xa0 \xa0,นิเชาว์ เอราวัณ ชนะคะแนน"\n'])

## `thaipbs-111k`

In [6]:
# !mkdir raw_data/thaipbs; mkdir cleaned_data/thaipbs-cleaned
# !wget https://www.dropbox.com/s/wer16m4l77sbx7x/thaipbs-111_final.csv; mv thaipbs-111_final.csv raw_data/thaipbs

In [11]:
thaipbs_df = pd.read_csv('raw_data/thaipbs/thaipbs-111_final.csv')
thaipbs_df.head()

,title,body,summary,labels,tags,url
0,นักวิจัยออสเตรเลียเผยสาเหตุฉลามโจมตีมนุษย์,นายนาธาน ฮาร์ท หัวหน้าทีมวิจัยจากมหาวิทยาลัยเว...,ทีมนักวิจัยออสเตรเลียศึกษาว่าเหตุใดฉลามถึงจู่...,ต่างประเทศ,"ฉลาม,นักวิจัย,มนุษย์,ออสเตรเลีย,แมวน้ำ,โจมตี",https://news.thaipbs.or.th/content/1
1,"""จีทีเอช"" สั่งพักงาน-งดใช้โซเชียลมีเดียชั่วครา...",เมื่อเวลาประมาณ 16.30 น.วันนี้ (8 เม.ย.2558) เ...,จีทีเอชและนาดาวบางกอก บริษัทต้นสังกัดนักแสดงซ...,ศิลปะ-บันเทิง,"GTH,จีทีเอช,นักแสดงฮอร์โมนเต้นในรถไฟฟ้าญี่ปุ่น...",https://news.thaipbs.or.th/content/5
2,"""ธชตวัน"" ประกาศลาออกจากกุนซือกัลฟ์ สระบุรี จาก...",การประกาศลาออกจากตำแหน่งกุนซือ กัลฟ์ สระบุรี เ...,ธชตวัน ศรีปาน ประกาศลาออกจากตำแหน่งหัวหน้าผู้...,กีฬา,"กัลฟ์ สระบุรี,กุนซือ,ธชตวัน,ธชตวัน ศรีปาน,ฟุตบ...",https://news.thaipbs.or.th/content/3
3,ตำรวจตุรกีจับผู้ต้องสงสัยยิงรถนักเตะสโมสรเฟเนบ...,เจ้าหน้าที่ตำรวจในประเทศตุรกียืนยันว่าได้จับกุ...,หลังเกิดเหตุลอบยิงรถนักเตะสโมสรเฟเนบาห์เชขณะเ...,กีฬา,"ข่าว16.00,ข่าวต่างประเทศ,ยิงรถสโมสร,ลีกตุรกี,เ...",https://news.thaipbs.or.th/content/4
4,สหรัฐฯประกาศสนับสนุนญี่ปุ่นและชาติพันธมิตรหากถ...,นายแอชตัน คาร์เตอร์ รัฐมนตรีว่าการกระทรวงกลาโห...,สหรัฐอเมริกาประกาศจุดยืนสนับสนุนญี่ปุ่นและชาต...,ต่างประเทศ,"Camp Schwab,ชินโสะ อาเบะ,ญี่ปุ่น,บารัค โอบามา,...",https://news.thaipbs.or.th/content/2


In [8]:
df = process_one(list(thaipbs_df.title))
df.text.to_csv('cleaned_data/thaipbs-cleaned/thaipbs_title.txt',header=None,index=False)
df.wc.describe()


7.0 23.0


count    91548.000000
mean        14.203926
std          4.853872
min          5.000000
25%         11.000000
50%         14.000000
75%         17.000000
max         73.000000
Name: wc, dtype: float64

In [16]:
df = process_one(list(thaipbs_df.body),sep_func=sent_tokenize)
df.text.to_csv('cleaned_data/thaipbs-cleaned/thaipbs_body.txt',header=None,index=False)
df.wc.describe()


3.0 181.0


count    674247.000000
mean         41.270462
std          55.937167
min           5.000000
25%          11.000000
50%          21.000000
75%          41.000000
max         300.000000
Name: wc, dtype: float64

In [13]:
df = process_one(list(thaipbs_df.summary))
df.text.to_csv('cleaned_data/thaipbs-cleaned/thaipbs_summary.txt',header=None,index=False)
df.wc.describe()


23.0 72.0


count    92131.000000
mean        43.744429
std         15.213230
min          8.000000
25%         33.000000
50%         42.000000
75%         52.000000
max        209.000000
Name: wc, dtype: float64

## `scb-mt-en-th-2020` (v1.0)


In [32]:
# !wget https://github.com/vistec-AI/dataset-releases/releases/download/scb-mt-en-th-2020_v1.0/scb-mt-en-th-2020.zip
# !unzip scb-mt-en-th-2020.zip -d raw_data/
# !rm scb-mt-en-th-2020.zip

In [36]:
fnames = glob.glob('raw_data/scb-mt-en-th-2020/*')
fnames

['raw_data/scb-mt-en-th-2020/thai_websites.csv',
 'raw_data/scb-mt-en-th-2020/generated_reviews_yn.csv',
 'raw_data/scb-mt-en-th-2020/wikipedia.csv',
 'raw_data/scb-mt-en-th-2020/mozilla_common_voice.csv',
 'raw_data/scb-mt-en-th-2020/generated_reviews_crowd.csv',
 'raw_data/scb-mt-en-th-2020/generated_reviews_translator.csv',
 'raw_data/scb-mt-en-th-2020/apdf.csv',
 'raw_data/scb-mt-en-th-2020/assorted_government.csv',
 'raw_data/scb-mt-en-th-2020/paracrawl.csv',
 'raw_data/scb-mt-en-th-2020/msr_paraphrase.csv',
 'raw_data/scb-mt-en-th-2020/task_master_1.csv',
 'raw_data/scb-mt-en-th-2020/nus_sms.csv']

In [39]:
dfs = []
for fname in fnames:
    dfs.append(pd.read_csv(fname))
df = pd.concat(dfs).reset_index(drop=True)
df.shape

(1001752, 2)

In [40]:
df.head()

,en_text,th_text
0,CLARINS This intensive replenishing cream hel...,คืนความยืดหยุ่นให้ผิวที่ร่วงโรยตามวัย จากการเป...
1,ONLY@CENTRAL Color : Almond Size : 32 A UK Ca...,ONLY@CENTRAL สี : เบจไซส์ : 32 A UK หมายเหตุ:...
2,ONLY@CENTRAL Color : Almond Size : 32 B UK Ca...,ONLY@CENTRAL สี : เบจไซส์ : 32 B UK หมายเหตุ:...
3,ONLY@CENTRAL Color : Almond Size : 32 C UK Ca...,ONLY@CENTRAL สี : เบจไซส์ : 32 C UK หมายเหตุ:...
4,ONLY@CENTRAL Color : Almond Size : 34 A UK Ca...,ONLY@CENTRAL สี : เบจไซส์ : 34 A UK หมายเหตุ:...


In [41]:
save_df = process_one(list(df.th_text))
save_df.wc.describe()


5.0 135.0


count    946775.000000
mean         31.396495
std          43.222842
min           5.000000
25%           9.000000
50%          15.000000
75%          29.000000
max         300.000000
Name: wc, dtype: float64

In [42]:
!mkdir cleaned_data/scb-mt-en-th-2020-cleaned
save_df.text.to_csv('cleaned_data/scb-mt-en-th-2020-cleaned/scb-mt-en-th-2020.txt',header=None,index=False)

## `tnc`

In [33]:
# !gsutil cp -r gs://aires_thai2transformers/data/raw_data/tnc raw_data
!mkdir cleaned_data/tnc-cleaned
!cd raw_data/tnc; ls | wc -l 

3417


In [38]:
fnames = glob.glob('raw_data/tnc/*')
len(fnames)

3417

In [44]:
dfs=[]
for fname in tqdm(fnames):
    with open(fname,'r') as f:
        x = [re.sub(r"<[^>]*>", "",i) for i in f.readlines()]
        dfs.append(process_one(x))
df = pd.concat(dfs).reset_index(drop=True)


0.0 73.0



0.0 113.49999999999969



0.0 181.54999999999995



0.0 122.5



0.0 75.04999999999981


...


0.0 43.5



0.0 108.74999999999999



0.0 145.25



0.0 161.54999999999993



0.0 171.25



In [45]:
df.wc.describe()

count    797390.000000
mean         46.886367
std          47.024226
min           5.000000
25%          15.000000
50%          30.000000
75%          61.000000
max         300.000000
Name: wc, dtype: float64

In [49]:
df.text.to_csv('cleaned_data/tnc-cleaned/tnc.txt',header=None,index=False)